In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
pip install gensim==3.8.1

  Using cached gensim-3.8.1-cp38-cp38-macosx_10_9_x86_64.whl
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
from konlpy.tag import Komoran
from nltk.probability import FreqDist
from gensim.models import Word2Vec

from pydub import AudioSegment
import numpy as np

from pydub import AudioSegment
import base64
import urllib3
import json

ModuleNotFoundError: No module named 'pydub'

# 나중에 추가할것

In [128]:
print(f"- 채널수: {audio_segment.channels}")
print(f"- 샘플 길이: {audio_segment.sample_width}")
print(f"- 프레임율(frame rate): {audio_segment.frame_rate}")
print(f"- 프레임폭(frame width): {audio_segment.frame_width}")
print(f"- 길이 (ms): {len(audio_segment)}")

- 채널수: 1
- 샘플 길이: 2
- 프레임율(frame rate): 16000
- 프레임폭(frame width): 2
- 길이 (ms): 30976


# 유창성

## 전처리

In [20]:
def processing_audio(audioFilePath):

  audio_segment = AudioSegment.from_mp3(audioFilePath)  # audio 파일을 ms단위로 분해
  audio_segment = audio_segment.set_frame_rate(16000)  # sampling rate 16000으로 설정
  audio_segment = audio_segment.set_channels(1) # channel을 1로 설정

  return audio_segment

## 침묵 구간 측정

In [76]:
def get_silence(user_answer, threshold, interval):
    
    # audio를 interval기준으로 into chunks로 분리
    # interval 1이면 1 m/s
    chunks = [user_answer[i:i+interval] for i in range(0, len(user_answer), interval)]
    
    # dBFS는 어떤 오디오 시스템이 클립핑(Clipping)이 발생하기 직전까지 사용할 수 있는 최대 신호의 크기
    # 임계값 보다 낮은 dBFS을 침묵 구간으로 측정
    silence_length = 0
    for chunk in chunks:
        if chunk.dBFS == float('-inf') or chunk.dBFS < threshold:
            silence_length += 1


    return silence_length

## 유창성 측정

In [6]:
def myrange(start, end, step):
    r = start
    while(r<end):
        yield r
        r += step

In [95]:
def score_fluency(audio):
    threshold = -80
    interval = 1
    user_silence = get_silence(audio, threshold, interval) # 사용자의 침묵 시간

    sec_rate = round(user_silence / len(audio_segment) * 100)
    rate_list = [i for i in range(33, 101, 1)]  # 침묵 시간 비율
    score_list = [round(s) for s in myrange(0, 100, round(100 / 67, 2))]
    score_list.reverse()
    score_dict = dict(zip(rate_list, score_list))  # 점수로 변환할 딕셔너리


    # 말한 시간의 침묵이 1/3정도는 사람이 듣기에 유창하므로 1/3보다 정적이 적으면 만점
    if sec_rate < 33: 
        score = 100
    else:
        score = score_dict[sec_rate]
        
    return score

# 발음평가

## 음성 분해

In [8]:
def segment(audio_segment,interval = 5000):
  chunks = [audio_segment[i:i+interval] for i in range(0, len(audio_segment), interval)]
  rawdatas = [chunk.raw_data for chunk in chunks]

  audioContents = []

  for chunk in chunks:
    rawdata= chunk.raw_data  # raw 데이터 추출
    audiocontent = base64.b64encode(rawdata).decode("utf8")  # 인코딩
    audioContents.append(audiocontent)

  return audioContents

## 음성 인식
모의고사가아닌 유형별 학습일 땐 정답있으니까 script에 정답 넣어주면 되겠다

In [9]:
def API(audioContent,script=None):
    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/PronunciationKor"
    accessKey = "ac679469-fbf1-4b08-abd7-f2aba1757ae6"
    languageCode = "korean"
    requestJson = {
        "access_key": accessKey,
        "argument": {
        "language_code": languageCode,
    #    "script" : script,
        "audio": audioContent
        }
    }

    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    js = response.data
    y = json.loads(js)
    user = y["return_object"]['recognized']
    score = y["return_object"]['score'] 

    return user,score

## 발음 평가 

In [10]:
def score_pronunciation(audioContents):
  
  user_answer = ''
  final_score = 0

  for audioContent in audioContents:
    user,score = API(audioContent)

    if user:
      user_answer += user
      final_score += score

  final_score  = round(final_score / len(audioContents)) * 20 

  return user_answer, final_score

# 토크나이징

In [11]:
def tokenizing(tokenizer, text):
    token = []
    all_token = []
    
    for sent in text.split('.'):
        morph = tokenizer.morphs(sent)
        if morph:
            token.append(morph)
            all_token += morph
    
    tagged = tokenizer.pos(text)
    nouns = [ word for word,pos in tagged if pos in ['NNG','NNP'] ]
    
    return token,nouns,all_token

# 키워드

In [12]:
def keyword(nouns):

    fdist = FreqDist(nouns)
    most_common = [token for token,freq in fdist.most_common(3)]
    
    return most_common

# 표현력

In [46]:
def expression(text, token, all_token):

    text_len = len(text.replace(' ','')) # 답안 길이
    word_len = len(set(all_token)) # 중복 제외 단어 수 
    
    # 토크나이징된 리스트에 대한 각 길이를 저장
    word_len_list = [len(t) for t in token]
    
    sent_len = len(token) # 문장 수
    word_len = sum(word_len_list) # 총 단어 수
    avg_len = word_len//sent_len #문장별 평균 단어 수

    return {'text_len': text_len,'word_len': word_len, 'sent_len': sent_len, 'avg_len':avg_len}

In [70]:
def score_expression(user_dict,answer_dict):
    text_len = round(user_dict['text_len'] / answer_dict['text_len'] * 100)
    word_len = round(user_dict['word_len'] / answer_dict['word_len'] * 100)
    sent_len = round(user_dict['sent_len'] / answer_dict['sent_len'] * 100) 
    avg_len = round(user_dict['avg_len'] / answer_dict['avg_len'] * 100)
    
    score = round(0.15 * (text_len +  sent_len) + 0.35 * (word_len + avg_len))
    
    if score > 100: # 사용자가 모범답안보다 문장,단어를 더 풍부하게 사용한 경우
        score = 100
        
    return score

# 모범답안 유사도

In [15]:
def text_similarity(user_all_token,answer_all_token):
    user = ' '.join(user_all_token)
    answer = ' '.join(answer_all_token)
    sent = (user,answer)
    count_vectorizer = CountVectorizer()
    count_matrix = count_vectorizer.fit_transform(sent)
    distance = cosine_similarity(count_matrix[0:1],count_matrix[1:2])[0][0] * 100
    
    return round(distance)

In [16]:
def score_similarity(user_all_token,user_nouns,answer_all_token,answer_nouns):
    all_sim = text_similarity(user_all_token,answer_all_token)
    nouns_sim = text_similarity(user_nouns,answer_nouns)
    
    return (all_sim + nouns_sim) //2 

# 주제의 연관성

In [17]:
def score_relevance(model,answer_keyword, user_keyword):
    sim_list = []
    
    for a in answer_keyword:
        for u in user_keyword:
            try:
                sim_list.append(model.wv.similarity(a,u))
            except KeyError:
                pass
            
    key_sim = text_similarity(user_keyword,answer_keyword)
    score = round((sum(sim_list) / len(sim_list))*50 + (key_sim*0.5))

    return score

# main

In [98]:
def evaluate(audio_file, answer):
    audio_segment = processing_audio(audio_file)
    audioContents = segment(audio_segment,interval = 5000)

    user,score = score_pronunciation(audioContents) 
    komoran = Komoran()
    model = Word2Vec.load('/content/ko.bin')
    
    user_token,user_nouns,user_all_token = tokenizing(komoran,user)
    answer_token,answer_nouns,answer_all_token = tokenizing(komoran,answer)
    user_dict = expression(user, user_token, user_all_token)
    answer_dict = expression(answer, answer_token, answer_all_token)

    answer_keyword = keyword(answer_nouns)
    user_keyword = keyword(user_nouns)
    
    flu = score_fluency(audio_segment)
    pro = score
    exp = score_expression(user_dict,answer_dict)
    sim = score_similarity(user_all_token,user_nouns,answer_all_token,answer_nouns)
    rel = score_relevance(model,answer_keyword, user_keyword)
    
    
    return dict(zip(['유창성','발음평가','표현력','유사도','주제의 연관성'],[flu,pro,exp,sim,rel]))

* 정답과 전혀 관련 없는 음성 test

In [99]:
audioFilePath = "/content/weekend_ordi.mp3"

answer = "제 취미는 영화 보기예요. 저는 시간 있을 때 영화관에 가요. 집에서도 영화를 자주 봐요. 저는 재미있는 영화를 좋아해요. 슬픈 영화도 잘 봐요. 저는 이번 주말에 친구와 함께 영화관에 갈 거예요."
evaluate(audioFilePath, answer)

{'발음평가': 60, '유사도': 12, '유창성': 94, '주제의 연관성': 4, '표현력': 100}

* 정답과 관련 있는 음성 test

In [100]:
audioFilePath = "/content/fast.mp3"

answer = "제 취미는 영화 보기예요. 저는 시간 있을 때 영화관에 가요. 집에서도 영화를 자주 봐요. 저는 재미있는 영화를 좋아해요. 슬픈 영화도 잘 봐요. 저는 이번 주말에 친구와 함께 영화관에 갈 거예요."
evaluate(audioFilePath, answer)

{'발음평가': 60, '유사도': 78, '유창성': 82, '주제의 연관성': 59, '표현력': 100}

* 정답과 연관은 있고, 짧고 빨리 말한 음성 test

In [101]:
audioFilePath = '/content/TEST1.mp3'

answer = "제 취미는 영화 보기예요. 저는 시간 있을 때 영화관에 가요. 집에서도 영화를 자주 봐요. 저는 재미있는 영화를 좋아해요. 슬픈 영화도 잘 봐요. 저는 이번 주말에 친구와 함께 영화관에 갈 거예요."
evaluate(audioFilePath, answer)

{'발음평가': 80, '유사도': 49, '유창성': 100, '주제의 연관성': 61, '표현력': 57}

# 문제점
* 표현력 점수 기준이 낮아서 그런지 100점이 자주나옴. 기준을 좀 높이기.
* 주제의 연관성이 연관성 있는 것에 비해서 너무 낮게나온다. 키워드가 일치하면 추가 점수를 주는 방향을 고려해보기.

# Ex

In [89]:
audioFilePath = "/content/fast.mp3"
#audioFilePath = "/content/weekend_ordi.mp3"
#audioFilePath = '/content/TEST1.mp3'
audio_segment = processing_audio(audioFilePath)
audioContents = segment(audio_segment,interval = 5000)
user,score = score_pronunciation(audioContents)

In [90]:
user

'대 치미 는 영화 보기 입 니다.저 는 시간 있 으면 영화관 에 가 요.저 는 영화 가 재밌 어야 좋 아 요.슬픈 영화 는 싫 어 해 요 저 는.친구 와 함께 영화관 에 갈 거 에 요.'

In [91]:
threshold = -80
interval = 1
user_silence = get_silence(audio_segment, threshold, interval)

In [96]:
score_fluency(audio_segment)

82

In [51]:
score

80

In [52]:
komoran = Komoran()
model = Word2Vec.load('/content/ko.bin')
user_token,user_nouns,user_all_token = tokenizing(komoran,user)
answer_token,answer_nouns,answer_all_token = tokenizing(komoran,answer)
user_dict = expression(user, user_token, user_all_token)
answer_dict = expression(answer, answer_token, answer_all_token)

answer_keyword = keyword(answer_nouns)
user_keyword = keyword(user_nouns)

In [53]:
user_nouns

['취미', '영화', '입', '니다', '시간', '영화관', '갑', '니다']

In [54]:
answer_nouns

['취미', '영화', '시간', '때', '영화관', '집', '영화', '영화', '영화', '이번', '주말', '친구', '영화관']

In [55]:
answer_keyword = keyword(answer_nouns)
user_keyword = keyword(user_nouns)
user_keyword

['니다', '취미', '영화']

In [56]:
answer_keyword

['영화', '영화관', '취미']

In [58]:
user_dict

{'avg_len': 9, 'sent_len': 2, 'text_len': 27, 'word_len': 19}

In [59]:
answer_dict

{'avg_len': 9, 'sent_len': 6, 'text_len': 81, 'word_len': 54}

In [75]:
score_expression(user_dict,answer_dict)

57

In [34]:
score_relevance(model,answer_keyword, user_keyword)

4

In [35]:
score_similarity(user_all_token,user_nouns,answer_all_token,answer_nouns)

12